### Contexte
Vous travaillez pour une entreprise de commerce électronique qui a collecté des données de ventes sur une année. Avant d'effectuer une analyse approfondie, l'entreprise souhaite s'assurer de la qualité et de la cohérence des données. Votre tâche est de créer un système de validation robuste en utilisant Pydantic v2.

### Objectifs
1. Créer des modèles Pydantic pour valider strictement les données de ventes.
2. Implémenter un système de validation qui identifie et catégorise les anomalies dans les données.
3. Générer un rapport détaillé sur la qualité des données.

### Données
Vous disposez d'un fichier CSV nommé "ventes_detaillees.csv", disponible en téléchargement sur [ce lien](http://sc-e.fr/wcs/ventes_detaillees.csv), contenant les colonnes suivantes :
id_vente, date, heure, id_produit, nom_produit, prix_produit, categorie_produit, quantite, pays, ville, methode_paiement, statut_commande, satisfaction_client

### Tâches

1. **Création des modèles Pydantic**
   - Créez un modèle `Produit` avec les contraintes suivantes :
     - `id`: doit commencer par 'P' suivi de 3 chiffres
     - `nom`: chaîne non vide
     - `prix`: nombre positif avec 2 décimales maximum
     - `categorie`: doit être l'une des catégories prédéfinies (Électronique, Vêtements, Alimentation, Maison, Loisirs, Beauté, Électroménager, Livres, Sport, Papeterie, Accessoires)

   - Créez un modèle `Vente` avec les contraintes suivantes :
     - `id`: doit commencer par 'V' suivi de 6 chiffres
     - `date` et `heure`: format valide
     - `produit`: utilise le modèle `Produit`
     - `quantite`: entier positif
     - `pays`: doit être l'un des pays européens prédéfinis (France, Allemagne, Espagne, Italie, Royaume-Uni, Belgique, Pays-Bas, Suisse, Portugal, Autriche)
     - `ville`: chaîne non vide
     - `methode_paiement`: doit être l'une des méthodes prédéfinies (Carte de crédit, PayPal, Virement bancaire, Apple Pay, Google Pay)
     - `statut_commande`: doit être l'un des statuts prédéfinis (En cours de traitement, Expédiée, Livrée, Annulée)
     - `satisfaction_client`: entier entre 1 et 5

2. **Système de validation**
   - Implémentez une fonction qui lit le CSV et valide chaque ligne.
   - Catégorisez les erreurs (par exemple : erreur de format, valeur hors limites, valeur manquante).
   - Comptabilisez les occurrences de chaque type d'erreur.

3. **Rapport de qualité des données**
   - Générez un rapport indiquant :
     - Le nombre total d'enregistrements traités
     - Le nombre d'enregistrements valides
     - Le nombre et le pourcentage d'enregistrements invalides
     - Un résumé des types d'erreurs rencontrées et leur fréquence
     - Des exemples d'enregistrements invalides pour chaque type d'erreur

### Code de départ

In [1]:
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List
from datetime import date, time
import pandas as pd

# Modèle Produit
class Produit(BaseModel):
    id: str = Field(..., pattern=r'^P\d{3}$')
    nom: str = Field(..., min_length=1)
    prix: float = Field(..., gt=0)
    categorie: str

    @field_validator('categorie')
    @classmethod
    def categorie_valide(cls, v):
        categories_valides = ['Électronique', 'Vêtements', 'Alimentation', 'Maison', 'Loisirs', 
                              'Beauté', 'Électroménager', 'Livres', 'Sport', 'Papeterie', 'Accessoires']
        if v not in categories_valides:
            raise ValueError(f"Catégorie invalide. Doit être l'une de : {categories_valides}")
        return v

# Modèle Vente
class Vente(BaseModel):
    id: str = Field(..., pattern=r'^V\d{6}$')
    date: date
    heure: time
    produit: Produit
    quantite: int = Field(..., gt=0)
    pays: str
    ville: str = Field(..., min_length=1)
    methode_paiement: str
    statut_commande: str
    satisfaction_client: int = Field(..., ge=1, le=5)

    @field_validator('pays')
    @classmethod
    def pays_valide(cls, v):
        pays_valides = ['France', 'Allemagne', 'Espagne', 'Italie', 'Royaume-Uni', 'Belgique', 
                        'Pays-Bas', 'Suisse', 'Portugal', 'Autriche']
        if v not in pays_valides:
            raise ValueError(f"Pays invalide. Doit être l'un de : {pays_valides}")
        return v

    @field_validator('methode_paiement')
    @classmethod
    def methode_paiement_valide(cls, v):
        methodes_valides = ['Carte de crédit', 'PayPal', 'Virement bancaire', 'Apple Pay', 'Google Pay']
        if v not in methodes_valides:
            raise ValueError(f"Méthode de paiement invalide. Doit être l'une de : {methodes_valides}")
        return v

    @field_validator('statut_commande')
    @classmethod
    def statut_commande_valide(cls, v):
        statuts_valides = ['En cours de traitement', 'Expédiée', 'Livrée', 'Annulée']
        if v not in statuts_valides:
            raise ValueError(f"Statut de commande invalide. Doit être l'un de : {statuts_valides}")
        return v

# Fonction pour valider les données
def valider_donnees_ventes(chemin_fichier: str) -> dict:
    data = pd.read_csv(chemin_fichier)
    erreurs = []
    valid_records = []
    for index, row in data.iterrows():
        try:
            produit = Produit(
                id=row['id_produit'],
                nom=row['nom_produit'],
                prix=row['prix_produit'],
                categorie=row['categorie_produit']
            )
            vente = Vente(
                id=row['id_vente'],
                date=pd.to_datetime(row['date']).date(),
                heure=pd.to_datetime(row['heure']).time(),
                produit=produit,
                quantite=row['quantite'],
                pays=row['pays'],
                ville=row['ville'],
                methode_paiement=row['methode_paiement'],
                statut_commande=row['statut_commande'],
                satisfaction_client=row['satisfaction_client']
            )
            valid_records.append(vente)
        except ValidationError as e:
            erreurs.append({'index': index, 'errors': e.errors()})
    return {'valid': valid_records, 'invalid': erreurs}

# Fonction pour générer un rapport de qualité
def generer_rapport_qualite(resultats_validation: dict):
    total = len(resultats_validation['valid']) + len(resultats_validation['invalid'])
    print(f"Nombre total d'enregistrements : {total}")
    print(f"Nombre d'enregistrements valides : {len(resultats_validation['valid'])}")
    print(f"Nombre d'enregistrements invalides : {len(resultats_validation['invalid'])}")
    
    # Fréquence des erreurs
    erreurs_par_type = {}
    for err in resultats_validation['invalid']:
        for e in err['errors']:
            type_erreur = e['type']
            if type_erreur not in erreurs_par_type:
                erreurs_par_type[type_erreur] = 0
            erreurs_par_type[type_erreur] += 1
    
    print("\nRésumé des erreurs :")
    for erreur, count in erreurs_par_type.items():
        print(f" - {erreur} : {count} occurrences")
    
    print("\nExemples d'erreurs :")
    for err in resultats_validation['invalid'][:5]:  # Afficher les 5 premières erreurs
        print(f"Index {err['index']} : {err['errors']}")

if __name__ == "__main__":
    chemin_fichier = "/Users/abedja/IaPM_WcS/Quetes/week4/Optionnelles/ventes_detaillees.csv"
    resultats = valider_donnees_ventes(chemin_fichier)
    generer_rapport_qualite(resultats)


Nombre total d'enregistrements : 10000
Nombre d'enregistrements valides : 9000
Nombre d'enregistrements invalides : 1000

Résumé des erreurs :
 - greater_than : 329 occurrences
 - value_error : 339 occurrences
 - string_pattern_mismatch : 160 occurrences
 - less_than_equal : 172 occurrences

Exemples d'erreurs :
Index 1 : [{'type': 'greater_than', 'loc': ('prix',), 'msg': 'Input should be greater than 0', 'input': -499.99, 'ctx': {'gt': 0.0}, 'url': 'https://errors.pydantic.dev/2.10/v/greater_than'}]
Index 13 : [{'type': 'value_error', 'loc': ('categorie',), 'msg': "Value error, Catégorie invalide. Doit être l'une de : ['Électronique', 'Vêtements', 'Alimentation', 'Maison', 'Loisirs', 'Beauté', 'Électroménager', 'Livres', 'Sport', 'Papeterie', 'Accessoires']", 'input': 'Catégorie Invalide', 'ctx': {'error': ValueError("Catégorie invalide. Doit être l'une de : ['Électronique', 'Vêtements', 'Alimentation', 'Maison', 'Loisirs', 'Beauté', 'Électroménager', 'Livres', 'Sport', 'Papeterie', '